# SkyPatrol event summary
Quick look notebook to summarize and triage `../../output/skypatrol_events.csv`.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)

CSV_PATH = '/home/calder/code/malca/output/skypatrol_events.csv'

df = pd.read_csv(CSV_PATH)
df


FileNotFoundError: [Errno 2] No such file or directory: '/home/calder/code/malca/output/skypatrol_events.csv'

In [ ]:
# Basic counts
total = len(df)
dip_sig = df['dip_significant'].sum()
jump_sig = df['jump_significant'].sum()
both_sig = ((df['dip_significant']) & (df['jump_significant'])).sum()
print(f'Total LCs: {total}')
print(f'Dip significant: {dip_sig}')
print(f'Jump significant: {jump_sig}')
print(f'Both significant: {both_sig}')
df[['dip_significant','jump_significant']].mean().to_frame('fraction')


In [ ]:
# Dip candidates sorted by Bayes factor (all rows)
dip_sorted = df.sort_values('dip_bayes_factor', ascending=False)
dip_cols = ['path','dip_significant','dip_best_morph','dip_best_delta_bic','dip_best_width_param',
            'dip_bayes_factor','dip_max_log_bf_local','dip_trigger_max','dip_max_event_prob']
dip_cols = [c for c in dip_cols if c in dip_sorted.columns]
dip_sorted[dip_cols]


In [ ]:
# Jump candidates sorted by Bayes factor (all rows)
jump_sorted = df.sort_values('jump_bayes_factor', ascending=False)
jump_cols = ['path','jump_significant','jump_best_morph','jump_best_delta_bic','jump_best_width_param',
             'jump_bayes_factor','jump_max_log_bf_local','jump_trigger_max','jump_max_event_prob']
jump_cols = [c for c in jump_cols if c in jump_sorted.columns]
jump_sorted[jump_cols]


In [ ]:
# Morphology distribution among significant dips
sig_dip = df[df['dip_significant']]
sig_jump = df[df['jump_significant']]
display(sig_dip['dip_best_morph'].value_counts())
display(sig_jump['jump_best_morph'].value_counts())


In [ ]:
# Quick scatter: max log BF vs bayes_factor for dips
fig, ax = plt.subplots(figsize=(6,4))
ax.scatter(df['dip_max_log_bf_local'], df['dip_bayes_factor'], s=12, alpha=0.5)
ax.set_xlabel('dip_max_log_bf_local')
ax.set_ylabel('dip_bayes_factor')
ax.set_title('Dip strength overview')
plt.show()


In [ ]:
# Full table of significant events (dip or jump)
sig = df[(df['dip_significant']) | (df['jump_significant'])].copy()
prob_cols = [c for c in ['dip_max_event_prob','jump_max_event_prob'] if c in sig.columns]
display(sig[['path','dip_significant','jump_significant'] + prob_cols])
sig


## More quick looks


In [ ]:
# Distribution of Bayes factors
fig, ax = plt.subplots(1, 2, figsize=(10,4))
ax[0].hist(df['dip_bayes_factor'], bins=30, color='steelblue', alpha=0.8)
ax[0].set_title('Dip Bayes factor')
ax[0].set_xlabel('dip_bayes_factor')
ax[0].set_ylabel('count')
ax[1].hist(df['jump_bayes_factor'], bins=30, color='darkorange', alpha=0.8)
ax[1].set_title('Jump Bayes factor')
ax[1].set_xlabel('jump_bayes_factor')
plt.tight_layout()
plt.show()


In [ ]:
# Dip vs jump Bayes factor scatter
fig, ax = plt.subplots(figsize=(5,5))
ax.scatter(df['jump_bayes_factor'], df['dip_bayes_factor'], s=12, alpha=0.6, 
           c=np.where(df['jump_significant'], 'darkorange', 'steelblue'))
ax.set_xlabel('jump_bayes_factor')
ax.set_ylabel('dip_bayes_factor')
ax.axhline(0, color='gray', lw=1, ls='--')
ax.axvline(0, color='gray', lw=1, ls='--')
ax.set_title('Dip vs Jump strength')
plt.show()


In [ ]:
# Run-level strength: duration vs sum for dips
fig, ax = plt.subplots(figsize=(6,4))
ax.scatter(df['dip_max_run_duration'], df['dip_max_run_sum'], s=12, alpha=0.6)
ax.set_xlabel('dip_max_run_duration (days)')
ax.set_ylabel('dip_max_run_sum')
ax.set_title('Dip run duration vs sum')
plt.show()


In [ ]:
# Summary stats for key numeric columns
cols = ['dip_bayes_factor','jump_bayes_factor','dip_max_run_points','jump_max_run_points',
        'dip_max_run_duration','jump_max_run_duration','dip_max_run_sum','jump_max_run_sum']
df[cols].describe().T


In [ ]:
# Event probability distributions (if present)
if 'dip_max_event_prob' in df.columns or 'jump_max_event_prob' in df.columns:
    fig, ax = plt.subplots(1, 2, figsize=(10,4))
    if 'dip_max_event_prob' in df.columns:
        ax[0].hist(df['dip_max_event_prob'].dropna(), bins=30, color='steelblue', alpha=0.8)
        ax[0].set_title('Dip max event prob')
        ax[0].set_xlabel('dip_max_event_prob')
    if 'jump_max_event_prob' in df.columns:
        ax[1].hist(df['jump_max_event_prob'].dropna(), bins=30, color='darkorange', alpha=0.8)
        ax[1].set_title('Jump max event prob')
        ax[1].set_xlabel('jump_max_event_prob')
    plt.tight_layout()
    plt.show()
else:
    print('No event probability columns found')


## Simulation: false-positive check
Generate synthetic light curves (noise-only and with injected dips) and run the Bayesian scorer to estimate false-positive rates.

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ensure imports work when running from notebooks/
candidates = [Path.cwd(), Path.cwd().parent, Path.cwd().parent.parent]
repo_root = next((p for p in candidates if (p / 'malca').exists()), Path.cwd())
for p in [repo_root, repo_root / 'malca']:
    sp = str(p.resolve())
    if sp not in sys.path:
        sys.path.insert(0, sp)

import events  # resolves to repo_root/malca/events.py

rng = np.random.default_rng(123)

def paczynski(t, t0, tE, amp):
    u = (t - t0) / max(tE, 1e-3)
    return -amp / np.sqrt(1.0 + u**2)

def drw_process(n, tau=50.0, sigma=0.05):
    dt = rng.exponential(1.0, size=n)
    x = np.zeros(n)
    for i in range(1, n):
        phi = np.exp(-dt[i]/tau)
        var = sigma**2 * (1-phi**2)
        x[i] = phi*x[i-1] + rng.normal(0, np.sqrt(var))
    return x

def simulate_lc(n_points=300, kind='noise', depth=0.4, width_days=5.0, amp=0.15, period=30.0, slope=0.0):
    jd = np.cumsum(rng.exponential(10.0, size=n_points)) + 2450000.0
    base_mag = 12.0
    err = rng.normal(0.03, 0.005, size=n_points).clip(0.01, 0.1)
    mag = rng.normal(base_mag, err)
    camera = np.zeros(n_points, dtype=int)

    if kind == 'dip':
        t0 = jd[rng.integers(10, n_points-10)]
        mask = np.abs(jd - t0) < width_days
        mag[mask] += depth
    elif kind == 'jump':
        t0 = jd[rng.integers(10, n_points-10)]
        mask = np.abs(jd - t0) < width_days
        mag[mask] -= depth
    elif kind == 'sin':
        phase = 2*np.pi*jd/period
        mag += amp*np.sin(phase)
    elif kind == 'rrab':
        phase = (jd/period) % 1
        template = np.where(phase < 0.2, -amp*(1-phase/0.2), amp*(phase-0.2)/0.8)
        mag += template
    elif kind == 'cepheid':
        phase = (jd/period) % 1
        mag += amp*np.sin(2*np.pi*phase) + 0.3*amp*np.sin(4*np.pi*phase)
    elif kind == 'eclipse':
        phase = (jd/period) % 1
        primary = np.abs(phase-0.0) < 0.05
        secondary = np.abs(phase-0.5) < 0.03
        mag[primary] += depth
        mag[secondary] += 0.5*depth
    elif kind == 'flare':
        t0 = jd[rng.integers(10, n_points-10)]
        mask = np.abs(jd - t0) < 1.0
        mag[mask] -= depth
    elif kind == 'microlens':
        t0 = jd[rng.integers(10, n_points-10)]
        mag += paczynski(jd, t0, width_days, depth)
    elif kind == 'drw':
        mag += drw_process(n_points, tau=50.0, sigma=0.08)
    elif kind == 'trend_sin':
        phase = 2*np.pi*jd/period
        mag += amp*np.sin(phase) + slope*(jd-jd[0])
    elif kind == 'bright_star':
        camera = rng.integers(0, 3, size=n_points)
        offsets = np.array([0.0, 0.08, -0.05])
        mag += offsets[camera]
        mag += rng.normal(0, 0.06, size=n_points)
        err = np.clip(err + rng.normal(0.01, 0.005, size=n_points), 0.005, 0.12)
    elif kind == 'bad_cluster':
        cluster_center = jd[rng.integers(n_points//4, 3*n_points//4)]
        cluster_mask = np.abs(jd - cluster_center) < 60
        mag[cluster_mask] += rng.normal(0.6, 0.1, size=cluster_mask.sum())
    elif kind == 'camera_offset':
        camera = rng.integers(0, 4, size=n_points)
        offsets = np.array([0.0, 0.15, -0.18, 0.08])
        mag += offsets[camera]
    elif kind == 'rcb':
        drop_starts = jd.min() + rng.uniform(150, 1700, size=2)
        for t0 in drop_starts:
            fade = (jd >= t0) & (jd <= t0 + 60)
            recovery = (jd > t0 + 60) & (jd <= t0 + 220)
            mag[fade] += 1.0 + 0.4 * rng.random(fade.sum())
            mag[recovery] += 1.0 * np.exp(-(jd[recovery] - (t0 + 60)) / 160)
    elif kind == 'contact_binary':
        p = period if period is not None else 0.5
        phase = (jd / p) % 1
        mag += amp * (0.7*np.sin(2*np.pi*phase) + 0.3*np.sin(4*np.pi*phase + 0.5))
    elif kind == 'semi_regular':
        p = period if period is not None else 120.0
        phase = 2*np.pi*jd/p
        envelope = 1.0 + 0.5*np.sin(2*np.pi*jd/(p*3))
        mag += amp * envelope * np.sin(phase) + rng.normal(0, 0.05, size=n_points)
    elif kind == 't_tauri':
        p = period if period is not None else 8.1
        phase = 2*np.pi*jd/p + rng.normal(0, 0.3, size=n_points)
        mag += amp * np.sin(phase) + rng.normal(0, 0.1, size=n_points)
    df = pd.DataFrame({'JD': jd, 'mag': mag, 'error': err})
    df['camera#'] = camera  # ensure per-camera baseline grouping works
    return df

def score_df(df):
    res = events.run_bayesian_significance(
        df,
        baseline_func=None,
        use_sigma_eff=False,
        require_sigma_eff=False,
        compute_event_prob=False,
        trigger_mode='logbf',
        logbf_threshold_dip=5.0,
        logbf_threshold_jump=5.0,
    )
    return {
        'dip_significant': res['dip']['significant'],
        'jump_significant': res['jump']['significant'],
        'dip_bayes_factor': res['dip']['bayes_factor'],
        'jump_bayes_factor': res['jump']['bayes_factor'],
        'dip_max_logbf': res['dip'].get('max_log_bf_local', np.nan),
        'jump_max_logbf': res['jump'].get('max_log_bf_local', np.nan),
    }

def assess(trials=50, scenario=None):
    rows = []
    sim_kwargs = {k: v for k, v in scenario.items() if k != 'name'}
    for _ in range(trials):
        df_sim = simulate_lc(**sim_kwargs)
        row = score_df(df_sim)
        row['scenario'] = scenario['name']
        row['truth_dip'] = scenario['kind'] == 'dip'
        row['truth_jump'] = scenario['kind'] == 'jump'
        rows.append(row)
    return pd.DataFrame(rows)

scenarios = [
    {'name': 'noise', 'kind': 'noise'},
    {'name': 'dip_shallow', 'kind': 'dip', 'depth': 0.2, 'width_days': 3.0},
    {'name': 'dip_deep', 'kind': 'dip', 'depth': 0.6, 'width_days': 5.0},
    {'name': 'jump_bright', 'kind': 'jump', 'depth': 0.3, 'width_days': 3.0},
    {'name': 'sinusoid', 'kind': 'sin', 'amp': 0.15, 'period': 30.0},
    {'name': 'rrab', 'kind': 'rrab', 'amp': 0.4, 'period': 0.8},
    {'name': 'cepheid', 'kind': 'cepheid', 'amp': 0.3, 'period': 5.0},
    {'name': 'eclipse', 'kind': 'eclipse', 'depth': 0.6, 'period': 3.0},
    {'name': 'flare', 'kind': 'flare', 'depth': 0.6, 'width_days': 0.5},
    {'name': 'microlens', 'kind': 'microlens', 'depth': 0.5, 'width_days': 5.0},
    {'name': 'drw', 'kind': 'drw'},
    {'name': 'trend_sin', 'kind': 'trend_sin', 'amp': 0.1, 'period': 40.0, 'slope': 0.0003},
    {'name': 'bright_nearby', 'kind': 'bright_star'},
    {'name': 'bad_point_cluster', 'kind': 'bad_cluster'},
    {'name': 'camera_offset', 'kind': 'camera_offset'},
    {'name': 'rcb', 'kind': 'rcb'},
    {'name': 'contact_binary', 'kind': 'contact_binary', 'period': 0.5, 'amp': 0.2},
    {'name': 'semi_regular', 'kind': 'semi_regular', 'period': 140.0, 'amp': 0.25},
    {'name': 't_tauri', 'kind': 't_tauri', 'period': 8.1, 'amp': 0.3},
]

results = []
for sc in scenarios:
    res = assess(trials=50, scenario=sc)
    results.append(res)
all_res = pd.concat(results, ignore_index=True)

# Detection rates by scenario
det_rates = all_res.groupby('scenario').agg(
    dip_det=('dip_significant','mean'),
    jump_det=('jump_significant','mean'),
    dip_bf_med=('dip_bayes_factor','median'),
    jump_bf_med=('jump_bayes_factor','median'),
)
display(det_rates)

# Bar plot of detection rates
fig, ax = plt.subplots(figsize=(10,4))
x = np.arange(len(det_rates))
ax.bar(x - 0.15, det_rates['dip_det'], width=0.3, label='dip_det', color='steelblue')
ax.bar(x + 0.15, det_rates['jump_det'], width=0.3, label='jump_det', color='darkorange')
ax.set_xticks(x)
ax.set_xticklabels(det_rates.index, rotation=30, ha='right')
ax.set_ylim(0,1.05)
ax.set_ylabel('Detection fraction')
ax.legend()
ax.set_title('Detection rates by scenario')
plt.tight_layout()
plt.show()

# Bayes factor distributions per scenario
ncols = 3
nrows = int(np.ceil(len(det_rates) / ncols))
fig, ax = plt.subplots(nrows, ncols, figsize=(5*ncols, 3*nrows))
ax = np.array(ax).ravel()
last = -1
for i, (sc, sub) in enumerate(all_res.groupby('scenario')):
    ax[i].hist(sub['dip_bayes_factor'], bins=30, alpha=0.7, color='steelblue', label='dip')
    ax[i].hist(sub['jump_bayes_factor'], bins=30, alpha=0.5, color='darkorange', label='jump')
    ax[i].set_title(sc)
    ax[i].set_xlabel('bayes_factor')
    ax[i].set_ylabel('count')
    ax[i].legend(fontsize=8)
    last = i
for j in range(last + 1, len(ax)):
    ax[j].axis('off')
plt.tight_layout()
plt.show()

# Plot one example from each scenario
nrows = int(np.ceil(len(scenarios) / ncols))
fig, ax = plt.subplots(nrows, ncols, figsize=(5*ncols, 3*nrows))
ax = np.array(ax).ravel()
last = -1
for i, sc in enumerate(scenarios):
    df_ex = simulate_lc(**{k:v for k,v in sc.items() if k!='name'})
    ax[i].errorbar(df_ex['JD'], df_ex['mag'], yerr=df_ex['error'], fmt='.', ms=3, alpha=0.6)
    ax[i].invert_yaxis()
    ax[i].set_title(sc['name'])
    last = i
for j in range(last + 1, len(ax)):
    ax[j].axis('off')
plt.tight_layout()
plt.show()

# Full results table
all_res


In [ ]:
# False positives: counts and rates by scenario and overall

def false_positive_stats(df):
    def agg(group):
        dip_neg = (~group["truth_dip"]).sum()
        jump_neg = (~group["truth_jump"]).sum()
        dip_fp = ((~group["truth_dip"]) & group["dip_significant"]).sum()
        jump_fp = ((~group["truth_jump"]) & group["jump_significant"]).sum()
        return pd.Series({
            "rows": len(group),
            "dip_fp": dip_fp,
            "dip_fp_rate": dip_fp / dip_neg if dip_neg else float('nan'),
            "jump_fp": jump_fp,
            "jump_fp_rate": jump_fp / jump_neg if jump_neg else float('nan'),
        })
    by_scenario = df.groupby("scenario").apply(agg)
    overall = agg(df).rename("overall")
    return by_scenario, overall

fp_by_scenario, fp_overall = false_positive_stats(all_res)
display(fp_by_scenario)
print("\nOverall false positives:")
display(fp_overall.to_frame().T)


## False positives on real Skypatrol light curves
Assess how often the pipeline flags dips/jumps on the real Skypatrol dataset using precomputed results.


In [ ]:
from pathlib import Path


def find_real_results():
    candidates = [
        Path('../../output/results_bayes.csv'),
        Path('../../output/skypatrol_events.csv'),
    ]
    if 'repo_root' in globals():
        candidates = [repo_root / p for p in candidates] + candidates
    for c in candidates:
        if c.exists():
            return c
    return None

REAL_RESULTS_PATH = find_real_results()
if REAL_RESULTS_PATH is None:
    raise FileNotFoundError("Could not find ../../output/results_bayes.csv or ../../output/skypatrol_events.csv; set REAL_RESULTS_PATH manually.")

df_real = pd.read_csv(REAL_RESULTS_PATH)
df_real['dip_or_jump'] = df_real['dip_significant'] | df_real['jump_significant']

def safe_mean(col):
    return float(df_real[col].mean()) if col in df_real else float('nan')

dip_rate = safe_mean('dip_significant')
jump_rate = safe_mean('jump_significant')
overall_rate = float(df_real['dip_or_jump'].mean())

print(f'Loaded {len(df_real)} real light curves from {REAL_RESULTS_PATH}')
print(f'Dip positives: {dip_rate:.3f}  Jump positives: {jump_rate:.3f}  Any-event: {overall_rate:.3f}')

metrics = {}
for col, key in [
    ('dip_bayes_factor', 'dip_bf_med'),
    ('jump_bayes_factor', 'jump_bf_med'),
    ('dip_max_log_bf_local', 'dip_logbf_med'),
    ('jump_max_log_bf_local', 'jump_logbf_med'),
]:
    if col in df_real:
        metrics[key] = df_real[col].median()

summary = pd.DataFrame([metrics])
display(summary)

cols_to_show = [c for c in [
    'path',
    'dip_significant', 'jump_significant',
    'dip_bayes_factor', 'jump_bayes_factor',
    'dip_max_log_bf_local', 'jump_max_log_bf_local',
    'baseline_source'
] if c in df_real]
sort_keys = [c for c in ['dip_bayes_factor','jump_bayes_factor'] if c in df_real]
top = df_real.sort_values(by=sort_keys, ascending=False) if sort_keys else df_real
display(top.head(5)[cols_to_show])


## Baseline inspection
Overlay each simulated light curve with the pipeline baseline (per_camera_gp_baseline) to spot baseline misfits.

In [ ]:
# Plot simulated light curves with GP baseline overlay (baseline ± sigma_eff)
import matplotlib.pyplot as plt
from malca.baseline import per_camera_gp_baseline
import events


def plot_baseline_grid(scenarios, ncols=3):
    n = len(scenarios)
    nrows = int(np.ceil(n / ncols))
    fig, axes = plt.subplots(nrows, ncols, figsize=(5*ncols, 3*nrows), sharex=False, sharey=False)
    axes = np.array(axes).ravel()
    for i, sc in enumerate(scenarios):
        df = simulate_lc(**{k: v for k, v in sc.items() if k != 'name'})
        try:
            df_base = per_camera_gp_baseline(df, **events.DEFAULT_BASELINE_KWARGS)
        except Exception as e:
            print(f"baseline failed for {sc['name']}: {e}")
            df_base = df.copy()
            df_base['baseline'] = np.nanmedian(df['mag'])
            df_base['sigma_eff'] = np.nanmedian(df['error'])
        ax = axes[i]
        ax.errorbar(df['JD'], df['mag'], yerr=df['error'], fmt='.', alpha=0.5, label='mag')
        if 'baseline' in df_base.columns:
            ax.plot(df_base['JD'], df_base['baseline'], color='crimson', lw=1.5, label='baseline')
        if 'sigma_eff' in df_base.columns:
            ax.fill_between(df_base['JD'], df_base['baseline'] + df_base['sigma_eff'],
                            df_base['baseline'] - df_base['sigma_eff'], color='crimson', alpha=0.15,
                            label='baseline ± sigma')
        ax.invert_yaxis()
        ax.set_title(sc['name'])
    # hide unused axes
    for j in range(i+1, len(axes)):
        axes[j].axis('off')
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper right')
    plt.tight_layout()
    plt.show()

plot_baseline_grid(scenarios)
